In [9]:
import params
import pathlib
import os
import wandb
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

wandb: Currently logged in as: gianfranco-romani. Use `wandb login --relogin` to force relogin


In [3]:
#raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
#path = Path(raw_data_at.download())

raw_data_at = run.use_artifact('gianfranco-romani/cisi-project/cisi_project:latest', type='dataset')
path = pathlib.Path(raw_data_at.download())

wandb:   5 of 5 files downloaded.  


In [24]:
#orig_questions = raw_data_at.get("questions")
#orig_queries = raw_data_at.get("queries")
#orig_mappings = raw_data_at.get("mappings")

In [11]:
t_mappings = raw_data_at.get("table_mappings")

wandb:   5 of 5 files downloaded.  


In [26]:
df = pd.DataFrame(data=t_mappings.data, columns=t_mappings.columns)
train, val, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])
print(len(t_mappings.data), len(train), len(val), len(test))

3114 1868 623 623


In [23]:
split_artifact = wandb.Artifact('data-splits', type='dataset', description='Train, validation, test dataset splits')

split_artifact.add(wandb.Table(dataframe=train), name='train-data')
split_artifact.add(wandb.Table(dataframe=val), name='val-data')
split_artifact.add(wandb.Table(dataframe=test), name='test-data')

<ManifestEntry digest: gRkUkNNGaAuMAQqzGRLwtg==>

In [25]:
run.log_artifact(split_artifact)
run.finish()